# 자연어처리 시작하기 - 전처리, 토큰화, 명사추출
* soynlp(https://github.com/lovit/soynlp)로 토큰화하고 명사를 추출해 워드클라우드 그려보기

In [ ]:
!pip install soynlp

In [ ]:
!pip show soynlp

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('foo1.csv', engine='python', encoding='utf-8')

In [3]:
sample_title = df['제목'][:2]
sample_title

0    "김학의 동영상, 3월 초 경찰이 제공"…시점·이유 규명해야
1    위기 내몰린 청와대 ‘메신저’…‘엇나간 해명’까지 사퇴 여론
Name: 제목, dtype: object

In [4]:
sample_content = df['본문'][:2]
sample_content

0    그런데 박지원 의원은 문제의 동영상을 지난 2013년 3월 초에 경찰에게 받아서 박...
1    SUB_TITLE_STARTSUB_TITLE_END문재인 대통령의 ‘입’인 김의겸 ...
Name: 본문, dtype: object

## 자신의 관심사에 맞는 단어로 데이터를 가져옵니다.

In [ ]:
p = r'.*(비행기|여행|언어|호텔).*'
care = df[df['제목'].str.match(p) |
           df['본문'].str.match(p, flags=re.MULTILINE)]
care.shape

In [ ]:
care.head()

In [ ]:
care.tail()
care.sample(10)

# 토큰화

In [ ]:
from soynlp.tokenizer import RegexTokenizer

tokenizer = RegexTokenizer()
tokenizer

In [ ]:
tokened_title = tokenizer.tokenize(sample_content)
tokened_title

# 텍스트 데이터 전처리 
* 개행문자 제거

In [ ]:
def preprocessing(text):
    # 개행문자 제거
    text = re.sub('\\\\n', ' ', text)
    return text

In [ ]:
# %time을 찍어주면 해당 코드를 실행할 때 걸리는 시간을 출력해 줍니다
%time sentences = df['본문'].apply(preprocessing)

In [ ]:
%time tokens = sentences.apply(tokenizer.tokenize)
tokens[:3]

In [ ]:
stopwords_kr = ['하지만', '그리고', '그런데', '저는','제가',
                '그럼', '이런', '저런', '합니다',
                '많은', '많이', '정말', '너무','[',']','것으로','했습니다','했다'] 

# 워드클라우드 그리기

In [ ]:
!pip install wordcloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline

def displayWordCloud(data = None, backgroundcolor = 'white', width=800, height=600 ):
    wordcloud = WordCloud(
                        font_path = '/Library/Fonts/NanumBarunGothic.ttf', 
                        stopwords = stopwords_kr,
                        background_color = backgroundcolor, 
                        width = width, height = height).generate(data)
    plt.figure(figsize = (15 , 10))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show() 

In [ ]:
# 결과를 출력해 보면 불용어(STOPWORD)가 너무 많습니다.
%time displayWordCloud(' '.join(sentences))

In [ ]:
from soynlp.noun import LRNounExtractor

In [ ]:
%%time
noun_extractor = LRNounExtractor(verbose=True)
noun_extractor.train(sentences)
nouns = noun_extractor.extract()

In [ ]:
# 추출된 명사를 찍어봅니다.
%time displayWordCloud(' '.join(nouns))

# 특정 이미지 형태로 워드 클라우드 그리기

In [ ]:
# 이미지 파일위에 출력하기
from PIL import Image
import numpy

In [ ]:
img = Image.open('apple.png')
img_array=numpy.array(img)

In [ ]:
wordcloud = WordCloud( font_path = '/Library/Fonts/NanumBarunGothic.ttf', 
                       stopwords = stopwords_kr,
                       background_color = 'white', 
                       mask=img_array,
                       width = 800, height = 600).generate(' '.join(nouns))
plt.figure(figsize = (15 , 10))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()  
wordcloud.to_file("simple.png")